In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
import os
import json

# Retrieve data

We begin by fetching the reduced data using the Python script `data_transfer.py` from the file `df_merged.parquet`, then importing it into Pandas data frame.

In [ ]:
df = pd.read_parquet('../scripts/df_GP_2nd_bug.parquet', engine='pyarrow')

In [ ]:
df.head(2)

In [ ]:
df.source.value_counts(normalize=True) * 100

In [ ]:
df.shape

Here we extract all unique IDs from our data frame:

In [ ]:
unique_ids = df['objectId'].unique().tolist()
len(unique_ids)

#

To calculate the weight values \(w_i\), we use the formula: `w_i` =\begin{cases}
\frac{1}{{\sigma_i^2}}, & \text{if data is available for day } i \\
0, & \text{otherwise}
\end{cases}

In [ ]:
missing_data = (df['dc_sigflux'] == 0)
df['dc_weight'] = np.where(missing_data, 0, 1 / (df['dc_sigflux'] ** 2))

In [ ]:
df[['source','dc_weight']].head(3)

# 

# 

# Percentenge of classes (classification) for all data 

The `Classifications_arch` folder stores the pre-computed classifications for each object in our datasets. These classifications are generated by the `Rn_jobs_th.py` script located in the `classification_functs` directory. Pre-computing the classifications saves time by storing the results for later analysis, making our workflow more efficient.

In [ ]:
# File path to unique IDs
ids_file_path = '../scripts/Classifications_arch/unique_ids.txt'

# Read the file and store each line as an element in a list
with open(ids_file_path, 'r') as file:
    IDS = file.read().splitlines()

# Initialize an empty list to store individual df_Classifications_arch
df_Classifications_arch = {}

# Folder name where the parquet files are located
folder_name = 'Classifications_arch' 

for Id in IDS:
    file_name = f'{Id}.parquet'
    file_path = f'../scripts/classification_functs/{folder_name}/{file_name}'

    # Check if the file exists
    if os.path.exists(file_path):
        # Read the Parquet file into a DataFrame
        data = pd.read_parquet(file_path)
        
        # Append the dataframe to the list
        df_Classifications_arch[Id] = data

df_Classifications_arch

# 

# Initialization of variables, tables ... 

We group the data by shared ID and create `NumPy` arrays for flux, weighted flux, and the source test(if it's a missing day(data)). We also determine the length of each time series.

In [ ]:
grouped = df.groupby('objectId')

F = grouped['dc_flux'].apply(lambda x: x.values)#.values
sig = grouped['dc_sigflux'].apply(lambda x: x.values)#.values
W = grouped['dc_weight'].apply(lambda x: x.values)#.values
source = grouped['source'].apply(lambda x: x.values)#.values
lengths = grouped['source'].apply(lambda x: len(x))#.values
mjd = grouped['mjd'].apply(lambda x: x.values)#.values


We define the length of our query,window or chunk, along with the limit factor and the size of each window or chunk.

In [ ]:
m = 0
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
chunk_size = 2 * (m + 1)

The `not_feasible_test` function determines the feasibility of each window. If the sum of values in each filter array exceeds half the total number of points in the filter, indicating all values are real, the function returns 1. Otherwise, it returns -1 to mark the window as 'not feasible'.

We set a rule to consider a window feasible if it has more than \(m/2\) for 'each' filter. This is just one way to do it; other conditions can be used depending on how much we trust the GPR model.

In [ ]:
half_pts = int((m+1)/2)
def not_feasible_test(array):
    if (array[::2].sum() > half_pts) & (array[1::2].sum() > half_pts) :
       return -1 
    return -99

"`objects`" list contains a subset of the objects we intend to work with.

"`L_max`" is defined to facilitate partial iteration, serving as a debugging aid by allowing a limit to be set on the number of iterations performed.

We initialize the NumPy arrays with `None` values.

#### Run this cell only once ! 

We shuffle our objects and save the shuffled data to ensure consistent results in subsequent analyses. For example, the value of `L_max` can vary slightly depending on the random shuffling of objects.

In [ ]:
# objects = unique_ids[:]#[0:10]
# num_objects = len(objects)
# # Shuffle the list
# random.shuffle(objects)
# # Save the shuffled list to a file
# file_path = 'shuffled_objects2.json'

# with open(file_path, 'w') as file:
#     json.dump(objects, file)

# print(f"Shuffled objects have been saved to {file_path}")

In [ ]:
# Path to the file
file_path = 'shuffled_objects.json'

# Read the shuffled list from the file
with open(file_path, 'r') as file:
    objects = json.load(file)

# print(objects)

`L_max` is the number of queries needed before every window gets at least one match. It depends on `m`, `num_objects`, and other factors. You can set the maximum number of windows to `L_max`. Initially, set `L_max` to 1 and compute the number of feasible windows below. After an initial trial, save `L_max` and return the code with the correct `L_max`. It's challenging, but here's the correct approach:

1. Initialize `L_max` to 1.
2. Compute the number of feasible windows :`count_W`. (before running the main algo)
3. Perform an initial trial with `L_max`=`count_W`.
4. Run the code and Save the value of `L_max`.
5. Return the code with the correct `L_max`.

In [ ]:
# objects = objects[:10]
num_objects = len(objects)

list_L_max = [97,1424,7221,7572,5701,3401,1801,297,10591] 
L_max = 97#10416#5154#5317#

print("L_max ", L_max)

R = np.empty(num_objects, dtype=object)
R_ref = np.empty(num_objects, dtype=object)
R_l = np.empty((num_objects, L_max), dtype=object)
num_by_obj = np.zeros((L_max,num_objects), dtype=object)
alp = np.empty((num_objects, L_max), dtype=object)
d = np.empty((num_objects, L_max), dtype=object)

In [ ]:
indexes_objects = range(num_objects)
Q = [None] * (L_max)
min_d =  [float('inf'), -1, -1] * (L_max)
num_left_by_Q = [0] * (L_max) ####### we don't need this one anymore <==> sum_i
selected_Q_K_i = np.empty(L_max, dtype=object)

We initialize R using the '`not_feasible_test`' function.

In [ ]:
start_time = time.time()
count_W = 0 
n = lengths[objects[0]]
num_chunks = int(n // 2)-m 

for k in indexes_objects:
    
    chunks = np.array([source[objects[k]][i*2 : (i*2+chunk_size)] for i in range(num_chunks)])
    result = np.array(list(map(not_feasible_test, chunks)))
    R[k] = result.copy()
    R_ref[k] = result.copy()
    
    count_W += np.count_nonzero(result == -1)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Num of feasible windows :",count_W)

print("Elapsed time:", elapsed_time/60, "minutes")

In [ ]:
indexes_array = np.array([np.where(array == -1)[0] for array in R], dtype=object)
rangek0 = [i for i in range(len(objects)) if len(indexes_array[i]) > 0]
len0 = len(rangek0)
len0

#

### We calculate the percentage of each class in the objects used prior to the first step (objects involved in the calculations)

In [ ]:
start_time = time.time()
classes_All = {}
for k in rangek0:
    df = df_Classifications_arch[objects[k]]
    for index, row in df.iterrows():
        class_name = row['classification']
        percentage = row['percentage']
        error_bar = row['error_bar']
        if class_name in classes_All:
            classes_All[class_name][0] += percentage
            classes_All[class_name][1] += error_bar**2
        else:
            classes_All[class_name] = [percentage,error_bar**2]

    # classes.append(get_classification3(objects[k]))
for class_name, list1 in classes_All.items():
    classes_All[class_name][0] = round(list1[0] / len0 , 2)
sorted_classes_All = dict(sorted(classes_All.items(), key=lambda item: item[1], reverse=True))

end_time = time.time()

print("Elapsed time:", (end_time - start_time)/60, "minutes")
sorted_classes_All

#

# Main algorithm : 

### Loop to compute the distance of the subsequence in the time series to its nearest neighbor.

In [ ]:
start_time = time.time()
rangek2 = list(range(len(objects)))

l= 0
while (l < L_max):
    indexes_array = np.array([np.where(array == -1)[0] for array in R], dtype=object)
    rangek2 = [i for i in range(len(objects)) if len(indexes_array[i]) > 0]

    has_empty_list = len(rangek2) == 0
    if has_empty_list:
        print("break , l = ", l )
        break
        
    num_left_by_Q[l] = len(rangek2)
        
    k = rangek2[0]
        
    f = F[objects[k]]
    index_no_match = indexes_array[k][0]
    k_Query_taked = k
    selected_Q_K_i[l] = [k,index_no_match]
    Q[l] = f[index_no_match*2 : index_no_match*2 +chunk_size]


    
    for k in rangek2:
        f = F[objects[k]]
        w = W[objects[k]]
        n = lengths[objects[k]]
        n_c = n - 2*m # (number of chunks x 2) ! it's (n/2 - m) but to optimize we mutiply by 2 directly !  



        s_1 = np.zeros(n_c, dtype=float)
        s_2 = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1): 
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:] += (f[j*2:j*2+ n_c]*h*w[j*2:j*2+ n_c])
            s_2[:] += (h**2 * w[j*2:j*2+ n_c])


        s_n = s_1[::2] + s_1[1::2]  
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        alp[k][l] = np.zeros_like(s_d, dtype=float)

        alp[k][l][mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero

        alpha = np.repeat(alp[k][l], 2) # duplicate alpha for each value (one for r and second for g)
     
    
        dd = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1):
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            
            dd[:] += ((f[j*2:j*2+ n_c] - alpha[:] * h)**2) * w[j*2:j*2+ n_c] 

        d[k][l] = dd[::2] + dd[1::2]
        
        factor_comparison =  d[k][l] <= factor
                
        R[k][indexes_array[k][factor_comparison[indexes_array[k]]]] = l # explanation follows below!
                
        R_l[k][l] = R_ref[k].copy()

        matches_k = float('inf')  # Start with a large number
        min_index = None  # Variable to store the index of the minimum value

        for i in range(len(factor_comparison)):
            if  R_l[k][l][i] != -99:
                if factor_comparison[i] :
                    R_l[k][l][i] = l
                    num_by_obj[l][k]+=1
                if index_no_match != i and d[k][l][i] < matches_k:
                        matches_k = d[k][l][i]  # Update the minimum value
                        min_index = i  #

        
        if matches_k < min_d[l * 3] :
            min_d[l * 3] = matches_k  # Instead of min_d[::3][l]
            min_d[l * 3 + 2] = min_index      # Instead of min_d[2::3][l]
            min_d[l * 3 + 1] = k            # Instead of min_d[1::3][l]

    R_l[k_Query_taked][l][index_no_match] = -2

 
    l += 1 
    
print("l = ",l)



end_time = time.time()

# Compute the elapsed time
elapsed_time += end_time - start_time

print("Elapsed time:", elapsed_time/60, "minutes")

Let's break down the expression `R[k][indexes_array[k][factor_comparison[indexes_array[k]]]] = l` step by step:

1. `indexes_array[k]`: This selects the array of indexes corresponding to the k-th element of `indexes_array`.
2. `factor_comparison[indexes_array[k]]`: This applies boolean indexing to `factor_comparison` using the indexes from `indexes_array[k]`. It selects only the elements of `factor_comparison` corresponding to the indexes in `indexes_array[k]`.
3. `indexes_array[k][factor_comparison[indexes_array[k]]]`: This gives the indices where the condition `factor_comparison` is true for the k-th element of `indexes_array`.
4. `R[k][indexes_array[k][factor_comparison[indexes_array[k]]]]`: This uses the indices obtained in the previous step to select elements from the k-th row of `R`.
5. `= l`: Finally, it assigns the value `l` to the selected elements of `R[k]`.


#

#

In [ ]:
num_left_by_Q[-1] # list of number of object for each step 

In [ ]:
plt.scatter(range(L_max),num_left_by_Q)

plt.xlabel('Queries')
plt.ylabel('Nb objs left')
plt.title('Number of objects left in function of queries.')


# Create DataFrame
# data = {'Queries': range(L_max), 'Nb objs left': num_left_by_Q}
# df = pd.DataFrame(data)

# # Export to Excel
# df.to_csv('num_left_by_Q.csv', index=False)

# Show the plot
plt.show()


#


# 

# 

### Calculate the Matrix of Matches MM (with windows represented in rows and queries as columns)

In [ ]:
rangek = range(len(objects))

# Initialize list to hold transposed arrays for each k
Matrices_by_k = []

no_missing = np.zeros(len(rangek), dtype= object)
length_no_missing = np.zeros(len(rangek), dtype= int)

Reference_Table_list = []

In [ ]:
start_time = time.time()
for k in rangek:
    no_missing[k] = np.where(R[k] != -99)[0]  
    
    length_no_missing[k] = len(no_missing[k])
    
    # Initialize transposed array
    Matrix= np.empty((length_no_missing[k], len(R_l[k])), dtype=object)
    
    refrence_table_k = np.empty(length_no_missing[k], dtype=object)
    
    # Transpose and store the arrays
    for i, idx in enumerate(no_missing[k]):
        for j, arr in enumerate(R_l[k]):
            if arr is not None:  # Check if arr is not None
                if arr[idx] == -2 or arr[idx] >= 0:
                    Matrix[i][j] = 1
                else:
                    Matrix[i][j] = 0
            else:
                Matrix[i][j] = 0
                #print(f"Error: arr is None at index {j} in R_l[{k}]")
            # if arr[idx] == -2 or arr[idx] >= 0:
            #     Matrix[i][j] = 1
            # else:
            #     Matrix[i][j] = 0
                
        refrence_table_k [i] =  [k ,idx]

                
    # Append transposed array to the list
    Matrices_by_k.append(Matrix)
    Reference_Table_list.append(refrence_table_k)

end_time = time.time()

# Compute the elapsed time
elapsed_time += end_time - start_time

print("Elapsed time:", elapsed_time/60, "minutes")
Matrices_by_k

In [ ]:
Matrix_Matches = np.concatenate(Matrices_by_k, axis=0)
Reference_Table = np.concatenate(Reference_Table_list, axis=0)
Matrix_Matches

In [ ]:
sum_j_rows = np.sum(Matrix_Matches, axis=1)
sum_j_rows


In [ ]:
sum_i_columns = np.sum(Matrix_Matches, axis=0)
sum_i_columns

#

#

### Functions that return some important information  : 

For a specific object k, this function returns the indexes of its feasible windows.

In [ ]:
def get_i_values_for_k(specific_k):
    return no_missing[specific_k]
get_i_values_for_k(0)  

#

For each row (window) in MM, this function returns the index information (the object number and the window index).

In [ ]:
def get_k_i_for_row(global_idx):
    return Reference_Table[global_idx]
k,i = get_k_i_for_row(2)
k,i

#

For a specific object k, this function returns the indexes of its feasible windows in the MM (index of rows)

In [ ]:
def get_i_indexes_for_k(specific_k):
    idx = 0 
    for k in rangek:
        if k == specific_k:
            return list(range(idx, idx + length_no_missing[k]))
        idx += length_no_missing[k]

get_i_indexes_for_k(0)

#

This function checks whether the window (in object `k_TS​` with index `i_TS`) has been marked as a query or not.

In [ ]:
def if_a_query(k_TS, i_TS, selected_Q_K_i): 
    mask = selected_Q_K_i != None
    for index_Q, (k, i) in enumerate(selected_Q_K_i[mask]):
        if k_TS == k and i_TS == i:
            return True, index_Q, f"This window is used as the {index_Q}th query! "
    return False, 0 , "This window is not used as a query ! "

get the number of match for a query i or a window j (they must be reversed ) 

In [ ]:
def get_i_by_sum(i):
        return sum_i_columns[i]

def get_j_by_sum(j):
        return sum_i_columns[j]

This function returns the object and the window (the origin) of the query.

In [ ]:
    
def get_k_i_of_Q(l):
    return selected_Q_K_i[l]
get_k_i_of_Q(0)

#

#

#

## Plotting functions  

In [ ]:
def plot_distance_flux(k,l,open_Fink= False):
    print("\n______________________________________________________________________________________________")
    print(f"{objects[k]}")
    #get_classification(objects[k])

    for index, row in df_Classifications_arch[objects[k]].iterrows():
        classification = row['classification']
        percentage = row['percentage']
        error_bar = row['error_bar']
        print(f"                        {classification} : {percentage:.1f}% ") 
   

    import ipywidgets as widgets
          
    button = widgets.Button(description=f"open in FINK")
    # Capture current values of kk and l in lambda's default arguments
    button.on_click(lambda _, k=k : on_button_clicked_plot(objects[k]))
    display(button)

    
    plt.figure(figsize=(14, 12))

    # Plot for F[k]
    plt.subplot(2, 1, 1)

    for i in range(int(len(F[objects[k]])/2)):
        if source.loc[objects[k]][2*i] == 0:
            marker = 'x'
            plt.errorbar(mjd.loc[objects[k]][2*i], F.loc[objects[k]][2*i], 
                     sig[objects[k]][2*i]*0.4,
                     c='C0', marker='x')
        else:
            marker = 'o'
                
            plt.errorbar(mjd.loc[objects[k]][2*i], F.loc[objects[k]][2*i], 
                     sig.loc[objects[k]][2*i],
                     c='C0', marker=marker)

        if source.loc[objects[k]][2*i+1] == 0:
            marker = 'x'
            plt.errorbar(mjd.loc[objects[k]][2*i+1], F.loc[objects[k]][2*i+1],
                     sig[objects[k]][2*i+1]*0.4,
                     c='C1', marker='x')
            
        else:
            marker = 'o'
                
            plt.errorbar(mjd.loc[objects[k]][2*i+1], F.loc[objects[k]][2*i+1],
                     sig.loc[objects[k]][2*i+1],
                     c='C1', marker=marker)
            
        a_query,l_1, string = if_a_query(k,i,selected_Q_K_i)
            
        if a_query and l_1 == l:
            print(string)
            
            # Define the window of indices
            window_start = mjd.loc[objects[k]][2*i]  # Index of the window start
            window_end = mjd.loc[objects[k]][2*i] + int(len(Q[l])/2 - 1)  # Index of the window end
            
            
            # Create an array of float indices
            indices = np.arange(window_start, window_end + 1)
            indices = np.concatenate(([indices[0] - 0.5], indices, [indices[-1] + 0.5]))

            # Plot a shaded region for the window
            plt.fill_between(indices, min(Q[l])/1.9, max(Q[l])*1.2, color='gray', alpha=0.2)
            
        
    plt.plot([], [], color='C1', marker='x', label='missing points !')
    plt.plot([], [], color='C0', marker='o', label='origin')
    plt.plot([], [], color='C0', marker='x', label='missing points !')
    plt.plot([], [], color='C1', marker='o', label='origin')
    
   
    ################################### what about the seccond window !? 
    moy_alp = 1

    plt.plot(mjd.loc[objects[k]][::2], F.loc[objects[k]][::2], c='C0', linewidth = 1)
    plt.plot(mjd.loc[objects[k]][1::2], F.loc[objects[k]][1::2], c='C1', linewidth = 1)

    plt.plot(np.arange(len(Q[l]) // 2) - 1-m + mjd.loc[objects[k]][0], Q[l][::2]*moy_alp, c='g', label='Q[l]',marker='.', linewidth=3, zorder=3)
    plt.plot(np.arange(len(Q[l]) // 2) - 1-m + mjd.loc[objects[k]][0], Q[l][1::2]*moy_alp, c='r', label='Q[l]',marker='.', linewidth=3, zorder=3)

    # Define the window of indices
    window_start = -1-m + mjd.loc[objects[k]][0] # Index of the window start
    window_end = int(len(Q[l])/2 - 1-1 -m)+ mjd.loc[objects[k]][0]  # Index of the window end

    # Create an array of float indices
    indices = np.arange(window_start, window_end + 1)
    indices = np.concatenate(([indices[0] - 0.4], indices, [indices[-1] + 0.4]))

    # Plot a shaded region for the window
    plt.fill_between(indices, min(Q[l])/1.2*moy_alp, max(Q[l])*1.2*moy_alp, color='gray', alpha=0.2)

    # plt.xlabel('Index')
    plt.ylabel('Flux')
    # plt.title('Flux Plot')
    plt.legend()
    




#     ###############                       Plot for d[k][0]                       ###############

    plt.subplot(2, 1, 2)
    plt.plot([-1-m, -1-m], [0, 0], color='none')  # Plot an empty line with zero length
    plt.plot(range(len(d[k][l])), d[k][l],color='C0', linestyle='-',linewidth=1)


    
    matches_idx = get_i_indexes_for_k(k)
    matches2 = sum_j_rows[matches_idx]


    # Plot dummy points with desired colors and markers
    plt.plot([], [], color='black', marker='o', label='Query chosed')#markers.soli
    plt.plot([], [], color='red', marker='x', label='missing cases !')
    plt.plot([], [], color='blue', marker='s', label='Matched here')#markers.ravioli
    plt.plot([], [], color='green', marker='*', label='Matches with a different `l` (Query)')#markers.stelline
    plt.plot([], [], color='yellow', marker='^', label=f'Not matched with any of the {L_max} options we selected') #markers.tortellini
    
    c= 0
    
    for i, val in enumerate(R_l[k][l]):
            if val == -99:
                plt.scatter(i, d[k][l][i], color='red', marker='x', s=50)  # marker size 50
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, 0, fontsize=10, color='red', ha='left')
                c+=1
#                 plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, 0*str(matches2[i]), fontsize=10, color='red', ha='left')

            elif val == l:
                plt.scatter(i, d[k][l][i], color='blue', marker='s', s=50)
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i-c]), fontsize=10, color='blue', ha='left')

            elif val == -2:
                plt.scatter(i, d[k][l][i], color='black', marker='o', s=50) 
                #print(matches2[i-c])
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i-c]), fontsize=10, color='black', ha='left')

            elif matches2[i-c]==0:
                plt.scatter(i, d[k][l][i], color='yellow', marker='^', s=50) 
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i-c]), fontsize=10, color='yellow', ha='left')

            else:
                plt.scatter(i, d[k][l][i], color='green', marker='*', s=75)
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i-c]), fontsize=10, color='green', ha='left')




    plt.legend(fontsize=8) 
    #plt.legend(loc='upper right', fontsize=8)

    plt.ylabel('distance')



    ###############                       Plot for alpha[k][l]                       ###############

#     plt.subplot(3, 1, 3)
#     plt.plot(range(len(alp[k][l])), alp[k][l], marker='.', linestyle='-',color='black')
#     plt.xlabel('Index')
#     plt.ylabel('Value')
#     plt.title('alpha Plot')




    plt.subplots_adjust(top=0.93)  # Adjust the top margin for the super title
    plt.suptitle(f"{objects[k]}, k = {k}, l = {l} ", fontname='Arial', fontsize=16, fontweight='bold')
    plt.show()
    

    if open_Fink:
        open_website(objects[k])


import webbrowser

def open_website(k):
    website_url = f'https://fink-portal.org/{k}'
    webbrowser.open(website_url)

def on_button_clicked_plot(Id):
    print("URL opened")
    open_website(Id)


#### This applies only to the closest distance for each query.

In [ ]:
def plot_distance_flux_min(k,l,min_d_i):
    print("\n______________________________________________________________________________________________")
    print(f"{objects[k]}")
    
    #get_classification(objects[k])

    for index, row in df_Classifications_arch[objects[k]].iterrows():
        classification = row['classification']
        percentage = row['percentage']
        error_bar = row['error_bar']
        print(f"                        {classification} : {percentage:.1f}% ") 
   
    
    import ipywidgets as widgets
          
    button = widgets.Button(description=f"open in FINK")
    # Capture current values of kk and l in lambda's default arguments
    button.on_click(lambda _, k=k : on_button_clicked_plot(objects[k]))
    display(button)

    
    plt.figure(figsize=(14, 12))

    # Plot for F[k]
    plt.subplot(2, 1, 1)
    for i in range(int(len(F[objects[k]])/2)):
        if source.loc[objects[k]][2*i] == 0:
            marker = 'x'
            plt.errorbar(mjd.loc[objects[k]][2*i], F.loc[objects[k]][2*i], 
                     sig[objects[k]][2*i]*0.4,
                     c='C0', marker='x')
        else:
            marker = 'o'
                
            plt.errorbar(mjd.loc[objects[k]][2*i], F.loc[objects[k]][2*i], 
                     sig.loc[objects[k]][2*i],
                     c='C0', marker=marker)

        if source.loc[objects[k]][2*i+1] == 0:
            marker = 'x'
            plt.errorbar(mjd.loc[objects[k]][2*i+1], F.loc[objects[k]][2*i+1],
                     sig[objects[k]][2*i+1]*0.4,
                     c='C1', marker='x')
            
        else:
            marker = 'o'
                
            plt.errorbar(mjd.loc[objects[k]][2*i+1], F.loc[objects[k]][2*i+1],
                     sig.loc[objects[k]][2*i+1],
                     c='C1', marker=marker)
            
        # a_query,l_1, string = if_a_query(k,i,selected_Q_K_i)
            
        # if a_query and l_1 == l:
        #     print(string)
            
            # Define the window of indices
    window_start = mjd.loc[objects[k]][2*min_d_i]  # Index of the window start
    window_end = mjd.loc[objects[k]][2*min_d_i] + int(m)  # Index of the window end
    
    
    # Create an array of float indices
    indices = np.arange(window_start, window_end + 1)
    indices = np.concatenate(([indices[0] - 0.5], indices, [indices[-1] + 0.5]))

    # Plot a shaded region for the window
    plt.fill_between(indices, min(F[objects[k]])/1.9, max(F[objects[k]])*1.2, color='gray', alpha=0.2)
        
    
    plt.plot([], [], color='C1', marker='x', label='missing points !')
    plt.plot([], [], color='C0', marker='o', label='origin')
    plt.plot([], [], color='C0', marker='x', label='missing points !')
    plt.plot([], [], color='C1', marker='o', label='origin')
    
   
    ################################### what about the seccond window !? 
    moy_alp = 1#+0*alp[k][l].mean()
    #moy_alp = np.median(F[k])/np.median(Q[l]) #Manu test

    plt.plot(mjd.loc[objects[k]][::2], F.loc[objects[k]][::2], c='C0', linewidth = 1)
    plt.plot(mjd.loc[objects[k]][1::2], F.loc[objects[k]][1::2], c='C1', linewidth = 1)

    plt.plot(np.arange(len(Q[l]) // 2) - 1-m + mjd.loc[objects[k]][0], Q[l][::2]*moy_alp, c='g', label='Q[l]',marker='.', linewidth=3, zorder=3)
    plt.plot(np.arange(len(Q[l]) // 2) - 1-m + mjd.loc[objects[k]][0], Q[l][1::2]*moy_alp, c='r', label='Q[l]',marker='.', linewidth=3, zorder=3)

    # Define the window of indices
    window_start = -1-m + mjd.loc[objects[k]][0] # Index of the window start
    window_end = int(len(Q[l])/2 - 1-1 -m)+ mjd.loc[objects[k]][0]  # Index of the window end

    # Create an array of float indices
    indices = np.arange(window_start, window_end + 1)
    indices = np.concatenate(([indices[0] - 0.4], indices, [indices[-1] + 0.4]))

    # Plot a shaded region for the window
    plt.fill_between(indices, min(Q[l])/1.2*moy_alp, max(Q[l])*1.2*moy_alp, color='gray', alpha=0.2)

    # plt.xlabel('Index')
    plt.ylabel('Flux')
    # plt.title('Flux Plot')
    plt.legend()

    
    plt.subplots_adjust(top=0.93)  # Adjust the top margin for the super title
    plt.suptitle(f"{objects[k]}, k = {k}, l = {l} ", fontname='Arial', fontsize=16, fontweight='bold')
    plt.show()
    


def on_button_clicked_plot(Id):
    print("URL opened")
    open_website(Id)


# 



# 



# Nearest query for each query : 


In [ ]:
nearst_query = {}
dis_to_nearst_query = []
for l, list_l in enumerate(selected_Q_K_i):
    #print(l, list)
    k,i = list_l
    
    nearst_query[l] = [-1,float('inf')]
    for l_after, list_after in enumerate(selected_Q_K_i[l+1:]):
        l_after += l+1
        # print(l, list,l_after, list_after)
        k_after, i_after = list_after
        if k_after != k :
            distance = d[k_after][l][i_after]
            # print(distance)
            if nearst_query[l][1] > distance :
                nearst_query[l] = [l_after,distance]
            
    for l_before, list_before in enumerate(selected_Q_K_i[:l]):
        # l_after += l+1
        # print(l, list,l_after, list_after)
        k_before, i_before = list_before
        if k_before != k :
    
            distance = d[k][l_before][i]
            # print(distance)
            if nearst_query[l][1] > distance :
                nearst_query[l] = [l_before,distance] 
    
    dis_to_nearst_query.append(nearst_query[l][1])

nearst_query


# 



# Nearst window(with only one match) to each query 



In [ ]:
nearst_window_1 = {}
dis_to_nearst_window_1 = [0]*L_max
for l, list_l in enumerate(selected_Q_K_i):
    # print(l, list_l)
    k,i = list_l
    
    nearst_window_1[l] = [-1,-1,float('inf')],[-1,-1,float('inf')],[-1,-1,float('inf')]
    # wind_l = list_l #get_k_i_of_Q(l)
    
    index = Reference_Table.tolist().index(list_l)#Reference_Table_list.index(wind_l)

    # print(l,"\n")
    
    for row, list_w in enumerate(Reference_Table[index+1:]):
        row += index +1
        k_window, i_window = list_w
        
        if sum_j_rows[row] == 1 and R[k_window][i_window] > l : 
            # print(index, row+index)
            # print(k_window, i_window)

            distance = d[k_window][l][i_window]
            # print(distance)

            if nearst_window_1[l][0][2] > distance :
                distance3 = nearst_window_1[l][1][2]
                distance2 = nearst_window_1[l][0][2]
                k_window3 = nearst_window_1[l][1][0]
                k_window2 = nearst_window_1[l][0][0]
                i_window3 = nearst_window_1[l][1][1]
                i_window2 = nearst_window_1[l][0][1]
                nearst_window_1[l] = ([k_window, i_window, distance],[k_window2, i_window2, distance2],[k_window3, i_window3, distance3])
    dis_to_nearst_window_1[l] = nearst_window_1[l][0][2]

  

nearst_window_1


# 


#

## plot histogram of numbers of matches for each Q

We graph the number of matches for each query.

In [ ]:
indexes = range(len(sum_i_columns))

# Plotting the histogram
plt.hist(indexes, bins=len(sum_i_columns), weights=sum_i_columns, color='C1', edgecolor='white')

# Adding labels and title
plt.xlabel('Indices')
plt.ylabel('Numbers')
plt.title('Histogram of numbers of matches for each Q ')

# Display the histogram
plt.show()


# Compute the highest and lowest 10 values and their indexes
sorted_indices = sorted(range(len(sum_i_columns)), key=lambda i: sum_i_columns[i])
lowest_10_indices = sorted_indices[:20]
highest_10_indices = sorted_indices[-10:][::-1]  # Reverse to get in descending order

total_windows = len(Matrix_Matches)

# Print the highest and lowest 10 values and their indexes with percentages
print("Highest 10:\n")
for i in highest_10_indices:
    percentage = (sum_i_columns[i] / total_windows) * 100
    print(f"Index: {i}, Matches: {sum_i_columns[i]}, Percentage: {percentage:.2f}%")

print("\n\nLowest 10:\n")
for i in lowest_10_indices:
    percentage = (sum_i_columns[i] / total_windows) * 100
    print(f"Index: {i}, Matches: {sum_i_columns[i]}, Percentage: {percentage:.2f}%")

    
    


# 

# 

# 

# 


# sort the queries(with one match) by the max nearst distance of each one 


First, we take the queries with only one match. Then, we sort these queries based on the distance to their nearest query, starting with the query that has the minimum distance to its nearest neighbor and ending with the query that has the maximum distance to its nearest neighbor.

In [ ]:
import numpy as np

# Convert sorted_indices and sum_i_columns to numpy arrays if they aren't already
sorted_indices = np.array(sorted_indices)
sum_i_columns = np.array(sum_i_columns)

# Use a boolean mask to find indices where sum_i_columns is less than or equal to 1
mask = sum_i_columns[sorted_indices] <= 1

# Get the corresponding indices
indices_1_match = sorted_indices[mask]


dis = np.array(dis_to_nearst_query) 

# Use indices_1_match to index dis
dis_1_match = dis[indices_1_match]

# Sort dis_1_match and get the sorted indices
sorted_dis_1_match_indices = np.argsort(dis_1_match)

# Sort indices_1_match based on sorted_dis_1_match_indices
sorted_indices_1_match = indices_1_match[sorted_dis_1_match_indices]#[::-1]

# Result: sorted indices of sum_i_columns based on sorted dis_1_match values
sorted_indices_1_match


In [ ]:
for i in sorted_indices_1_match:
    print(i, nearst_query[i])

#

# Save information to an Excel file for archiving and analysis

In fact, this can help reduce the number of queries with only one match by transforming Excel data into a table format, then selecting specific interesting classes for analysis (and put them in the list `specific_classes`).

In [ ]:
import pandas as pd


data = []

for l in sorted_indices:

    Q_ratio = (Q[l][::2]).sum()/(Q[l][1::2]).sum() / (m + 1)
    k, i = selected_Q_K_i[l]
    non_zero_indexes = np.nonzero(num_by_obj[l])[0]
    #Nb_valid = np.sum(is_valid[objects[k]][i*2:i*2+chunk_size])
    url_and_matches = []
    for kk in non_zero_indexes:
        url_and_matches.append(f"https://fink-portal.org/{objects[kk]} : Nb of matches : {num_by_obj[l][kk]}")
    
    string = "" 
    for index, row in df_Classifications_arch[objects[k]].iterrows():
        classification = row['classification']
        percentage = row['percentage']
        error_bar = row['error_bar']
        string += f"  {classification} : {percentage:.1f}% \n"
        
    # Create a dictionary for each iteration
    row = {
        'Query': l,
        'Matches on this Query': sum_i_columns[l],
        'Ratio': Q_ratio,
        'Object of this Q': objects[k],
        'Link': f"https://fink-portal.org/{objects[k]}",
        'class': string,
        #'Nb_valid' : Nb_valid,
        'mjd ': mjd.loc[objects[k]][2*i], 
        'Annotation': '',
        #'min d (nearst window 1 match)': nearst_window_1[l][0][2],
        'Number of objects': len(non_zero_indexes),
        'Objects with matches': non_zero_indexes.tolist(),
        'URLs and Matches': '\n'.join(url_and_matches)
    }
    
    # Append the dictionary to the data list
    data.append(row)
    
    if sum_i_columns[l] >1 : # we only choose queries with one match
        break 
    

df = pd.DataFrame(data)

# Export DataFrame to Excel
df.to_excel(f'lowest_Q_EM_GP_m{m}_o1-2_real_@.xlsx', index=False)


In [ ]:
df

#

# Visual Analysis for Results: 

In [ ]:
specific_classes = [7715,7716,7987,9113,9114,9520,9566,9567,9568,9569,4845]

`sorted_indices_1_match[::-1]:` It sorts the queries from the most unusual (compared to the others) to the most typical.

In [ ]:
print("###################  m = ", m ,"######################")
count= 0 
for l in sorted_indices_1_match[::-1]:#sorted_indices:#sorted_indices_1_match[::-1]:#sorted_indices:#[0:1549]:
    k , i = selected_Q_K_i[l]
    Nb_valid = np.sum(source[objects[k]][i*2:i*2+chunk_size])

    #if Nb_valid < 8: # We can select queries where all the data is either real or has few missing values.
    #    continue
    if sum_i_columns[l] >1 :
            break

    count+=1
    
    print("-----------------------------------------------------------------------------------------------------------",l,"\n")
    print("----------------------------------------NEW OBJECT --------------------------------------------------------\n")
    print("-----------------------------------------------------------------------------------------------------------\n")
    print(f"Q[{l}] , Matches on this Query: {sum_i_columns[l]}")
    
    # print('Nb_valid', Nb_valid)

    

    
    print("Ratio :",Q[l][0]/Q[l][1])
    
    print(f"Object of this Q : {objects[k]} : https://fink-portal.org/{objects[k]}")

    
    non_zero_indexes = np.nonzero(num_by_obj[l])[0]
    non_zero_indexes, len(non_zero_indexes)
    print("\nNumber of objects that have matches on this Q :", len(non_zero_indexes))
    print("Objects that have matches on this Q : ", non_zero_indexes,"\n")
    
    for kk in non_zero_indexes:
        print(f"https://fink-portal.org/{objects[kk]}" , " : Nb of mathces : ",num_by_obj[l][kk])
        button = widgets.Button(description=f"plot {objects[kk]}")
        # Capture current values of kk and l in lambda's default arguments
        button.on_click(lambda _, kk=kk, l=l: on_button_clicked(kk, l))
        display(button)
        
    print()
    
    plot_distance_flux(k,l)

    print("-----------------------------------------------------------------------------------------------------------\n")
    #nearst_w_k, nearst_w_i, nearst_w_d = nearst_window_1[l][0]

    #print(nearst_w_i, nearst_w_d)
    
    #plot_distance_flux(nearst_w_k,l)

    """print("------------------------------- nearst query --------------------------------------------")

    l_nearst, d_nearst = nearst_query[l]
    k_nearst , i_nearst = selected_Q_K_i[l_nearst]
    
    if l_nearst < l:
        plot_distance_flux_min(k_nearst,l_nearst,i_nearst)
    else:
        plot_distance_flux_min(k_nearst,l,i_nearst)"""

    
    """print("------------------------------- nearst window with one match --------------------------------------------")

    k_nearst , i_nearst, d_nearst = nearst_window_1[l][0]
    # k_nearst , i_nearst = selected_Q_K_i[l_nearst]
    print(i_nearst)
    print("Distance = ",d_nearst)
    
    # plot_distance_flux(k_nearst,l)
    plot_distance_flux_min(k_nearst,l,i_nearst)
    

    print("------------------------------- nearst window --------------------------------------------")

    min_d_d  = min_d[3*l]
    min_d_k = min_d[3*l+1]
    min_d_i = min_d[3*l+2]
    
    if min_d_d != float('inf') : 
        # print(min_d_d, min_d_k,min_d_i)
        print("distance :",min_d_d)
        print("K :", min_d_k)
        print("mjd :", mjd[objects[k]][min_d_i*2])
        # plot_distance_flux(min_d_k,l)
        plot_distance_flux_min(min_d_k,l,min_d_i)
    else : 
        print("no min found")
        print("---------------------------------------------------------------------------")"""

    
    if count == 50 :
     break

In [ ]:
count,m

#

#

# Percentage of classes  without repetition

In [ ]:
rangek3 = [] # unique objects with 1 match (as query ) 
for l in sorted_indices:
    if sum_i_columns[l] >1 : ################ i only choose queries with one match ( for m=0 we only have 4 !!!)
        break 
    k, i = selected_Q_K_i[l]
    if k not in rangek3:
        rangek3.append(k)

In [ ]:
classes_wr = {}
c= len(rangek3)

for k in rangek3 :
    df = df_Classifications_arch[objects[k]]  # Example call, modify as necessary
    
    # Iterate over the rows of the DataFrame
    for index, row in df.iterrows():
        class_name = row['classification']
        percentage = row['percentage']
        error_bar = row['error_bar']
        
        if class_name in classes_wr:
            classes_wr[class_name][0] += percentage
            classes_wr[class_name][1] += error_bar**2
        else:
            classes_wr[class_name] = [percentage,error_bar**2]

    # classes.append(get_classification3(objects[k]))
for class_name, list1 in classes_wr.items():
    classes_wr[class_name][0] = round(list1[0] / c , 2)
    classes_wr[class_name][1] = np.sqrt(list1[1]) / c
sorted_classes_Anm = dict(sorted(classes_wr.items(), key=lambda item: item[1], reverse=True))

sorted_classes_Anm

In [ ]:
nb_classes = 10 
all_objects = dict(list(sorted_classes_All.items())[:nb_classes])
anomalous_objects =  dict(list(sorted_classes_Anm.items())[:nb_classes])
# Extracting data for plotting
classes = list(all_objects.keys())
percentages_all = [all_objects[cls][0] for cls in classes]
errors_all = [all_objects[cls][1] for cls in classes]

percentages_anomalous = [anomalous_objects.get(cls, [0, 0])[0] for cls in classes]
errors_anomalous = [anomalous_objects.get(cls, [0, 0])[1] for cls in classes]

# Plotting
x = np.arange(len(classes))
width = 0.35  # Width of the bars

fig, ax = plt.subplots(figsize=(12, 8))

# Plot for all objects
bars_all = ax.bar(x - width/2, percentages_all, width, yerr=errors_all, label='All Objects', capsize=5)

# Plot for anomalous objects
bars_anomalous = ax.bar(x + width/2, percentages_anomalous, width, yerr=errors_anomalous, label='Anomalous Objects', capsize=5)

# Adding labels and title
ax.set_xlabel('Classes')
ax.set_ylabel('Percentage')
ax.set_title('Class Distribution Comparison Without repetition')
ax.set_xticks(x)
ax.set_xticklabels(classes, rotation=45, ha='right')
ax.legend()

# Adding percentage labels on top of the bars
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

add_labels(bars_all)
add_labels(bars_anomalous)

plt.tight_layout()
plt.show()


#

#

#

#